## Æther Composition Priority
| z-index | Group | Item(s) |
| --- | --- | --- |
| 0 | Background | Background color |
| 1 | Background | Tilemap |
| 2 | Interaction | Non-player sprites |
| 3 | Interaction | Player sprite |
| 4 | Interaction | Projectiles |
| 5 | Effects | Visual effects |
| 6 | Overlay | UI |


## Steps:

### Layer 0: Background 
---
- **Goal: Print a solid color background to the terminal display.**
  - [x] Print a single color to the terminal.
  - [x] Fills the terminal by default.
  - [ ] ~~Never exceeds terminal bounds~~
  - [x] Can be resized smaller than the terminal
  - [x] Use 256-color extended ANSI codes.
  - [x] Keep a `term_utils.py` module for common terminal operations.
- **Classes Needed**
  - [x] NyxEntity
  - [x] NyxEntityManager
  - [x] NyxComponentStore
  - [x] NyxComponent (ABC)
  - [x] BackgroundColorComponent
  - [x] NyxSystem (ABC)
  - [x] RendererSystem
  - [x] AetherCompositor
  - [x] HemeraRenderer

- After a few trials, working up through the composition layers seems the best way to approach the problem. 
- So far so good through the `NyxSystem` class. Next is working on rendering collection.
- Render system collects and sends the prioritized entity dict to Aether
- Basic HemeraRenderer created, rendering a pixel to the screen

Next: Aether -> Hemera

### Layer 1: Tilemap 
---
- **Goal**
  - [x] Print an array of tiles to the terminal
  - [x] Fill zeros with bg color
  - [x] Cache/buffer tilemap after compute
  - [x] Store tilemaps in a resource or store by ID
- **Classes Needed**
  - [x] TilesetStore
  - [x] TilemapComponent
  - [x] Tilemap
  - [x] TilemapSystem
  - [x] SceneComponent
  - [x] PositionComponent

- Decided to move `Store`-type dictionaries to class attributes for easy access. `TilesetStore` has been made with class attributes.
- SceneComponent allows for a global container/flag
- PositionComponent should allow for scrolling

### 12/7/24 - Tag v0.0.1-alpha issued: 
---
- Render a color frame to the terminal. Basic ECS -> Render -> Output 

### Intermediate Goal: .nyx file imports
---
- **Goal**
  - [x] Allow importing .nyx assets from files. This is essentially an array that gets read and converted into an ndarray.

- Working, though inherently insecure import module created. Will require significant hardening down the line, but works for the PoC stage of the program.

### Intermediate Goal: Subpixel Rendering
---
- **Goal**
  - [x] Update HemeraTermFx to use subpixel drawing
  - [x] Adjust the current calculations to match the doubled height resolution.
  - [x] Print 256-color tilemap in subpixels to the terminal. 

- Used simple list iterations for the actual printing, for now.
- Explore if its easier or faster to slice the ndarray before iterating.

### 12/8/24 - Tag v0.0.2-alpha issued:
--- 
- Render a Tilemap to the terminal in color using subpixel rendering. 

### Intermediate Goal: Delta Framebuffer Rendering
---
- **Goal**
  - [x] Update HemeraTermFx to generate a delta framebuffer of only changed pixels.
  - [x] Sparcely print the delta framebuffer by skipping any unchanged pixels.
  - [x] Ensure color data is not lost (ie, carry over unchanged subpixel color data if its paired counter part (bg if fg, fg if bg color) changes).
  - [x] Uses stacked NumPy matrices (2, y, x) to track fg and bg color in x, y space.
  - [x] Use `sys` module for printing.
  - [x] Do not exceed terminal bounds when printing.

- Some debugging nightmares occured with misaligned axes, cursor relocations inaccuracy, and lost subpixel color data. This was mainly due to reversed axis order in the 3D array as well as some other, minor errors. Tracing the anomalous printing patterns back to the errors was a significant undertaking given the inherent challenge in visualizing the data in ways outside of the rendering pipeline.
- The 3D matrix is iterated as if it were a 2D matrix by creating a 2D index ndarray and then iterating that instead.

### Intermediate Goal: Refactoring and Docstrings
---
- **Goal**
  - [x] Provide complete docstrings in every module.
  - [x] Update file structure and naming.
  - [x] Break up AetherRenderer in a main and multiple utility classes.
  - [x] Create and update the changelog.
  - [x] Update the readme.

- Mostly involved clarifying variable names, enshrining workflow and process information, etc. Plus added the mythology sections to explain the naming scheme.
- Nearly ready for first release (v0.0.3-alpha)

### Layer 2: Nonplayer Sprites
---
- **Goal**
  - [x] Create a sprite and place it in an x, y coordinate within the frame
  - [x] Use `PositonComponent` to guide location
  - [x] Ensure lower layers are visible in transparent areas of sprite (not overwritten)


### Intermediate Goal: Terminal Printing Optimization
---
- **Goal**
  - [x] Use line profiling to identify bottlenecks in the rendering pipeline.
  - [x] Aggresively optimize the printing pipeline. 

- **Results**
  
  | Version | Total Time | Per Frame Time | FPS |
  |--------|--------|--------|--------|
  | **v0.1.1-alpha** | **4.64321 s** | **0.0141 s** | **70.9** |
  | v0.1.0-alpha | 10.1012 s | 0.0307 s | 32.6 |
  | v0.0.4-alpha | 40.4568 s | 0.1230 s | 8.1 |

- **Notes**
  - Unexpectedly, the bottleneck was not in the actual terminal printing, but in the string buffer generation.
  - Optimization has taken the time to render 329 frames from 40.4568 seconds to 4.64321 seconds, a 88.5% reduction in time or a **10.9x** speedup in frame printing.
    - Removed tuple generation within the loop
    - Removed string concatenation within the loop
    - Used a sum function to combine fg and bg color ints to check for changes in both colors while only fetching one value.
    - The summed pixels are then summed into row sums, which are used to skip rows that have not changed.
    - Split the sum array and the fg color arrays into separate arrays to allow for different data types.
    - Use StringIO to buffer the string after accumulating the row strings.
    - Only fetch fg color if first the row and then the pixel has changed.
    - Use the sum and the fg color to compute the original bg color without having to fetch it from the array/memory.
    - Ensured all variables are preallocated and precomputed before the loop, or at least instantiated outside of the loop.
    - Generate and store two dictionaries at runtime to map the fg and bg colors to their respective full ANSI escape strings instead of generating them within the loop.


### Intermediate Goal: Working Demo
---
- **Goal**
  - [x] Create a working demo of the rendering/printing pipeline.
  - [x] Allow for the easy printing of images, gifs, and videos to the terminal.
  - [x] Import NPZ files for easy image loading.
  - [ ] Create an easy conversion tool for images to NPZ files.
  - [x] Create block letters for the terminal when zoomed out and the font is too small to read otherwise.

In [ ]:
import numpy as np

new_subpixel_frame = np.ones((10, 10), dtype=np.uint8)
old_frame = np.eye(10, dtype=np.uint8)
delta_buffer = np.where(new_subpixel_frame != old_frame, new_subpixel_frame, 0)

print(old_frame)
print(delta_buffer)


[[1 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 1]]
[[0 1 1 1 1 1 1 1 1 1]
 [1 0 1 1 1 1 1 1 1 1]
 [1 1 0 1 1 1 1 1 1 1]
 [1 1 1 0 1 1 1 1 1 1]
 [1 1 1 1 0 1 1 1 1 1]
 [1 1 1 1 1 0 1 1 1 1]
 [1 1 1 1 1 1 0 1 1 1]
 [1 1 1 1 1 1 1 0 1 1]
 [1 1 1 1 1 1 1 1 0 1]
 [1 1 1 1 1 1 1 1 1 0]]


In [93]:
# Subpixel and Delta testing
import numpy as np

old_frame = np.array([
    [0 for _ in range(5)],  # fg color
    [5 for _ in range(5)],  # bg color
    [11 for _ in range(5)], # fg color
    [16 for _ in range(5)], # bg color
    [22 for _ in range(5)], # fg color
    [27 for _ in range(5)], # bg color
    [33 for _ in range(5)], # fg color
    [38 for _ in range(5)], # bg color
    [44 for _ in range(5)], # fg color
    [49 for _ in range(5)], # bg color
], dtype=np.uint8)

new_frame = np.arange(50).reshape(10, 5) # Mock frame


old_subpixel_frame = np.stack([old_frame[::2, :], old_frame[1::2, :]], axis=1) # split fg and bg to make 3d array
new_subpixel_frame = np.stack([new_frame[::2, :], new_frame[1::2, :]], axis=1) # split fg and bg to make 3d array

# Keep new_subpixel_frame fg & bg pixel at a given x, y index if EITHER fg OR bg color had changed from the same z-pair at the same x, y coordinates in the old_subpixel_frame
# If both pairs are the same, replace with zeros

delta_frame = np.where(
    np.any(new_subpixel_frame != old_subpixel_frame, axis=1, keepdims=True),
    new_subpixel_frame,
    np.zeros_like(new_subpixel_frame)
)
print(delta_subpixel_frame)


# Mock slice:
old_frame_pair = np.array([16, 17])

# Get subpixel pair = ([[16, 17]])
new_frame_pair = new_subpixel_frame[1:2:, 1:2:, 1:3] 

# Compare. Returns True if ANY change
changed = np.any(new_frame_pair != old_frame_pair)
print(f"Changed = {changed}")

[[[ 0  1  2  3  4]
  [10  0 12 13 14]
  [20 21  0 23 24]
  [30 31 32  0 34]
  [40 41 42 43  0]]

 [[ 0  6  7  8  9]
  [15  0 17 18 19]
  [25 26  0 28 29]
  [35 36 37  0 39]
  [45 46 47 48  0]]]
Changed = False


In [ ]:
from datetime import datetime
from typing import List
from multiprocessing import Pool, cpu_count

# A CPU-bound task (e.g., squaring a large range of numbers)
def square_number(n):
    return n * n

if __name__ == "__main__":
    # Input data
    numbers: List(int) = list(range(1, 100000000))
    start = datetime.now()
    # Create a pool of workers equal to the number of CPU cores
    with Pool(cpu_count()) as pool:
        # Map the function to the data
        results = pool.map(square_number, numbers)
    stop = datetime.now()
    delta = stop - start
    print(delta)

In [ ]:
from datetime import datetime
# A CPU-bound task (e.g., squaring a large range of numbers)
def square_number(n):
    return n * n

if __name__ == "__main__":
    # Input data
    numbers: List(int) = list(range(1, 100000000))
    start = datetime.now()
    results = [square_number(number) for number in numbers]
    stop = datetime.now()
    delta = stop - start
    print(delta)

NameError: name 'List' is not defined

In [19]:
from datetime import datetime

# Function to check if a point (c) is in the Mandelbrot set
def mandelbrot(c, max_iter=1000):
    z = 0 + 0j
    for n in range(max_iter):
        z = z * z + c
        if abs(z) > 2:
            return n
    return max_iter

if __name__ == "__main__":
    # Define the dimensions of the grid and the range of complex numbers
    width, height = 800, 800
    x_min, x_max = -2.0, 1.0
    y_min, y_max = -1.5, 1.5
    max_iter = 1000

    # Start timer
    start = datetime.now()

    # Sequential calculation of the Mandelbrot set
    result = []
    for i in range(height):
        row = []
        for j in range(width):
            x = x_min + (x_max - x_min) * j / (width - 1)
            y = y_min + (y_max - y_min) * i / (height - 1)
            c = complex(x, y)
            row.append(mandelbrot(c, max_iter))
        result.append(row)

    # Stop timer
    stop = datetime.now()
    delta = stop - start
    print(f"Sequential execution time: {delta}")


Sequential execution time: 0:00:09.466620


In [20]:
from datetime import datetime
from multiprocessing import Pool

# Function to check if a point (c) is in the Mandelbrot set
def mandelbrot(c, max_iter=1000):
    z = 0 + 0j
    for n in range(max_iter):
        z = z * z + c
        if abs(z) > 2:
            return n
    return max_iter

# Function to calculate a row of the Mandelbrot set
def calculate_row(i, width, x_min, x_max, y_min, y_max, max_iter):
    row = []
    for j in range(width):
        x = x_min + (x_max - x_min) * j / (width - 1)
        y = y_min + (y_max - y_min) * i / (height - 1)
        c = complex(x, y)
        row.append(mandelbrot(c, max_iter))
    return row

if __name__ == "__main__":
    # Define the dimensions of the grid and the range of complex numbers
    width, height = 800, 800
    x_min, x_max = -2.0, 1.0
    y_min, y_max = -1.5, 1.5
    max_iter = 1000

    # Start timer
    start = datetime.now()

    # Create a pool of workers
    with Pool() as pool:
        # Parallel calculation of the Mandelbrot set, distributing rows across workers
        result = pool.starmap(calculate_row, [(i, width, x_min, x_max, y_min, y_max, max_iter) for i in range(height)])

    # Stop timer
    stop = datetime.now()
    delta = stop - start
    print(f"Parallel execution time: {delta}")


Parallel execution time: 0:00:01.671719
